### OSM til bygg‑polygoner med DuckDB (uten ekstra verktøy)

Denne notebooken viser, steg for steg, hvordan vi kan lese en OSM PBF‑fil (Norge) direkte i DuckDB, forstå strukturen i OSM‑dataene, og bygge reelle byggepolygoner – uten verktøy som Imposm.

Hva vi gjør:
- Laster ned PBF for Norge
- Leser PBF direkte i DuckDB med spatial‑utvidelsen
- Får et raskt overblikk over OSM‑modellen (noder, veier, relasjoner, tags)
- Rekonstruerer byggepolygoner fra OSM «ways»
- Transformerer til metersystem og beregner areal i m²

Kort om OSM (enkelt forklart):
- OSM er et åpent kart dugnad: alle kan bidra.
- Hvert kartobjekt har:  
  - «node» (punkt),  
  - «way» (en ordnet liste med noder – brukes for linjer og flater),  
  - «relation» (knytter sammen flere elementer, f.eks. komplekse flater med hull).
- «Tags» er nøkkel=verdi‑par (f.eks. `building=yes`, `name=Skole`).

Hva er PBF?
- Kompakt binært format for OSM‑data (kjappere og mindre enn XML).
- Vi bruker Geofabrik sin ferdiglagde PBF for Norge.
- Lisens: ODbL (må kreditere OSM‑bidragsytere ved bruk).

Slik jobber vi i resten av notebooken:
1) Last ned PBF  
2) Sett opp DuckDB + spatial (valgfritt Azure)  
3) En enkel tabell med bygg‑tagger (uten geometri)  
4) Bygg polygoner fra «ways» ved å slå opp noder og lage flater  
5) Transformér til et metersystem og regn ut areal i m²



In [2]:
import os

import duckdb
from dotenv import load_dotenv
import urllib.request as res

### Steg 1: Last ned PBF for Norge

- Kilde: Geofabrik leverer ferske OSM‑uttrekk som PBF‑filer per land/region.  
- Vi laster kun ned hvis filen ikke finnes fra før (spar tid og båndbredde).  
- Resultatet er en lokal fil `norway-latest.osm.pbf` som vi kan lese direkte i DuckDB.



In [3]:
url = "https://download.geofabrik.de/europe/norway-latest.osm.pbf"
local_file = "norway-latest.osm.pbf"

if not os.path.exists(local_file):
    print("Downloading Norway OSM data...")
    res.urlretrieve(url, local_file)
    print("Download complete.")

### Steg 2: Sett opp DuckDB + spatial (og valgfritt Azure)

- Vi bruker DuckDB i minne (raskt å komme i gang).
- Vi installerer/loader `spatial`‑utvidelsen for å lese OSM direkte med `ST_ReadOSM(...)`.
- Vi installerer/loader `azure` (valgfritt) for å kunne lese/lagre data i Azure Blob.  
  Tilkoblingsstreng hentes fra `.env` og settes som DuckDB‑parameter.



In [4]:
load_dotenv()
connection_string = os.getenv("BLOB_STORAGE_CONNECTION_STRING")

con = duckdb.connect(":memory:")
con.execute("INSTALL spatial; LOAD spatial;")
con.execute("INSTALL azure; LOAD azure;")
con.execute("SET azure_storage_connection_string = ?;", [os.getenv("BLOB_STORAGE_CONNECTION_STRING")])
con.execute("SET azure_transport_option_type = curl")

### Steg 3: Første blikk på bygg (uten geometri)

- Vi spør etter objekter med `tags['building']` og lager en tabell `buildings`.
- Dette gir oss metadata (type bygg, andre tags), men ikke geometri enda.
- OSM‑objekter kan være `node`, `way` eller `relation`.  
  I neste steg bygger vi selve polygonene ved å bruke noder som «byggesteiner».



In [5]:
con.execute(
    """
    CREATE OR REPLACE TABLE buildings AS
    SELECT id,
           kind,
           tags['building'] AS building_type,
           tags
    FROM ST_ReadOSM('norway-latest.osm.pbf')
    WHERE tags['building'] IS NOT NULL
    """)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

### Steg 4: Bygg polygoner fra OSM «ways»

Slik lager vi flater av bygg:
- De fleste bygg i OSM er `way` med en ordnet liste av node‑IDer (`refs`).
- Vi flater ut denne listen med rekkefølge (ordinalitet), og slår opp `lat`/`lon` for hver node.
- Vi lager punkter og binder dem sammen til en linje (`ST_MakeLine(...)`).
- Vi tar bare lukkede linjer (`ST_IsClosed`) og gjør dem om til polygon (`ST_MakePolygon`).
- Resultatet blir byggepolygoner med tilhørende tagger (f.eks. `building`, `name`, `operator`).

### Steg 5: Koordinatsystem og areal i m²

- OSM‑koordinater er i WGS84 (`EPSG:4326`), som er i grader.
- For areal i m² trenger vi meter: vi transformerer til UTM sone 33N (`EPSG:25833`).
- Da kan vi beregne areal i m² med `ST_Area(...)` på den transformerede geometrien.
- Merk: Norge dekker flere UTM‑soner. For hele landet kan 25833 gi litt større feil i ytterkantene. For høy presisjon kan man dele opp per sone eller bruke en likeareal‑projeksjon.



In [ ]:
con.sql("""CREATE OR REPLACE TABLE buildings AS
WITH building_ways AS (
    SELECT
        osm.id AS way_id,
        osm.tags,
        t.node_id,
        t.idx
    FROM ST_ReadOSM('norway-latest.osm.pbf') AS osm,
    UNNEST(osm.refs) WITH ORDINALITY AS t(node_id, idx)
    WHERE
        osm.kind = 'way'
        AND osm.tags['building'] IS NOT NULL
),
node_points AS (
    SELECT
        id AS node_id,
        lat,
        lon
    FROM ST_ReadOSM('norway-latest.osm.pbf')
    WHERE kind = 'node'
),
way_geometries AS (
    SELECT
        bw.way_id,
        bw.tags,
        ST_MakeLine(
            LIST(ST_Point(np.lon, np.lat) ORDER BY bw.idx)
        ) AS linestring
    FROM building_ways bw
    JOIN node_points np ON bw.node_id = np.node_id
    GROUP BY bw.way_id, bw.tags
),
polygon_geometries AS (
    SELECT
        way_id,
        tags,
        ST_MakePolygon(linestring) AS geometry
    FROM way_geometries
    WHERE ST_IsClosed(linestring)
)
SELECT
    way_id,
    tags['building'] AS building_type,
    tags['industrial'] AS industrial_type,
    tags['name'] AS name,
    tags['operator'] AS operator,
    tags['start_date'] AS start_date,
    tags['description'] AS description,
    ST_Transform(geometry, 'EPSG:4326', 'EPSG:25833') AS geometry_25833,
    ST_Area(ST_Transform(geometry, 'EPSG:4326', 'EPSG:25833')) AS area_m2
FROM polygon_geometries;""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

### Tips, begrensninger og videre arbeid

- Komplekse bygg kan være `relation` (multipolygon) med hull. Denne oppskriften dekker hovedsakelig lukkede `ways`.
- Ikke alle `ways` er lukket; vi filtrerer dem bort for å unngå ugyldige polygoner.
- Store PBF‑filer kan bruke tid/minne. Vurder mindre regioner eller filtrering.
- Azure: siden utvidelsen er lastet og tilkobling satt, kan man skrive/laste direkte mot Blob ved behov (f.eks. `COPY`/`CREATE TABLE` til `azure://...`).
- Videre arbeid:
  - Støtte `relation`‑baserte multipolygoner (ytre/indre ringer).
  - Lagre resultat som Parquet/GeoParquet for deling og videre analyse.
  - Enkle analyser: største bygg per kommune, fordeling av `building`‑typer, osv.

